<a href="https://colab.research.google.com/github/c-c-c-c/100knocks/blob/master/Neural_Style_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>tf.KerasによるNeural Styleの実装（勾配法バージョン）</center>

Neural Style Transferの理解に行く前に、まずはニューラルネットの定める画風 <b>neural style</b> 、CNNによるテクスチャ合成についてよく理解してみましょう。
Neural Style Transferの解説

https://colab.research.google.com/github/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb#scrollTo=fwzYeOqOUH9_

とできるだけ似せて作ります(スタイルだけ計算するので、NSTよりはシンプルです)。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. 学習済みVGG19とその特徴マップ

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

vgg_model = VGG19(weights='imagenet', include_top=False)

In [ ]:
print(len(vgg_model.layers), 'layers')
vgg_model.layers

これら22層のうち、[Gatys et al., ]3.2節に書いてあるように、以下では‘conv1 1’, ‘conv2 1’, ‘conv3 1’, ‘conv4 1’, ‘conv5 1’の5層を用います。ですので、その層のインデックスをリストで用意しておきましょう。

In [ ]:
style_layer_indexes = [1, 4, 7, 12, 17]

この5個のVGG19の特徴マップを計算するモデルを、`feature_model`という名前で用意しておきます。

In [ ]:
from tensorflow.keras.models import Model

inputs = vgg_model.layers[0].input
features = []
for i in style_layer_indexes:
    features.append(vgg_model.layers[i].output)

feature_model = Model(inputs=inputs, outputs=features)
# このモデルのパラメータ（VGGのパラメータ）は学習しない！
feature_model.trainable = False

これで画像を入力することで、‘conv1 1’, ‘conv2 1’, ‘conv3 1’, ‘conv4 1’, ‘conv5 1’の5層に対する特徴マップが出力となるモデルができました。

In [ ]:
feature_model.summary()

適当なディレクトリに、スタイル画像を保存しておきましょう。今回はスタイルとして、原論文にも多用されていたゴッホの星月夜を選びました：

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = '/content/drive/My Drive/style/Gogh.jpg'
img = image.load_img(img_path, target_size=(512, 512))
# 画像を表示
img

この画像を、VGG19（正確には先ほど作った`feature_model`）に入力できるように前処理を行う。今回はVGGを無視して、[-1,1]にピクセル値を規格化する

In [ ]:
def preprocess_input(raw_img):
    x = raw_img.copy()
    x = x / (255/2) -1
    x = x.astype('float32')
    return x

def deprocess_img(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                                 "dimension [1, height, width, channel] or [height, width, channel]")
    if len(x.shape) != 3:
        raise ValueError("Invalid input to deprocessing image")
    x = (x+1) * (255/2)
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# imageオブジェクトをnumpy ndarrayへ変換
x_gogh = image.img_to_array(img)
# x[np.newaxis,:,:,:]と同じ。一枚でも計画「行列」化
x_gogh = np.expand_dims(x_gogh, axis=0)
# VGG専用の前処理関数を使う。1/255ではない
x_gogh = preprocess_input(x_gogh)

では実際に`feature_model`に入力して、5種類の特徴マップそれぞれの初めのチャネルだけを可視化してみましょう：

In [ ]:
features_gogh = feature_model.predict(x_gogh)

for f in features_gogh:
    plt.imshow(f[0,:,:,0])
    plt.show()

以下では、これら各層の特徴マップのチャネル相関（Gram行列）が担う星月夜のスタイル情報を可視化することにしましょう。

# 2. Neural Style計算の実装

このノートブックではNSTの前に、その元となった発見(Texture Synthesis Using Convolutional Neural Networks)

https://papers.nips.cc/paper/5633-texture-synthesis-using-convolutional-neural-networks.pdf

を勉強します。

まず、特徴マップ（ndarrayじゃなくてTFのテンソルオブジェクト）からグラム行列を計算する関数、特徴マップとスタイル画像のグラム行列から二乗誤差を計算する関数、生成画像の滑らかさを測るTV誤差関数、そして全体の誤差を計算する関数を順次実装する：

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K 

def gram_matrix(f):
    # f: feature map. channel last, shape = (1, h, w, c)
    #_, h, w, c = tf.shape(f).numpy()
    _, h, w, c = K.int_shape(f)
    gram = tf.einsum('nijc,nijd->cd', f+0.1, f+0.1)# グラム行列がスパースになるのを防ぐのに、ベースの値を少し上げる
    return gram 

def get_style_loss(feature, gram_target):
  # feature: feature map of generated image. channel last, shape = (1, h, w, c)
  # gram_target: gram matrix of a feature map for a given target style image
  _, h, w, c = K.int_shape(feature)
  gram = gram_matrix(feature)
  return tf.reduce_sum(tf.square(gram - gram_target))/ (tf.cast(4. * (h**2) *(w**2), tf.float32))

def total_variation_loss(x):
    # x: image. channel last, shape = (1, H, W, 3)
    a = K.square(x[:,  :-1, :, :] - x[:, 1:, :, :])
    b = K.square(x[:,  :, :-1, :] - x[:, :, 1:, :])
    return tf.reduce_mean(a) + tf.reduce_mean(b)

def compute_loss(model, image, style_features, n_layer, lambda_tv, lambda_loss):
    # image: generated image
    # style_features: list of style feature maps of intermediate layers
    # n_layer: the index of the layer of your interesrt
    img_features = model(image)  
    img_feature = img_features[n_layer]

    style_feature = style_features[n_layer]
    gram_target = gram_matrix(style_feature)

    style_loss =  get_style_loss(img_feature, gram_target)
    tv_loss = total_variation_loss(image)
  
    return  style_loss * lambda_loss  + tv_loss * lambda_tv

とりあえず学習途中で止めてもいいように、次のリストに計算結果を貯めていきましょう（このやり方良くないので、ちゃんとスマートに変えてください）

In [ ]:
imgs = []

では、実際に最適化でニューラルスタイルを計算する関数を実装します：

In [ ]:
def run_neural_styler(style_path, model, n_iterations=1000, n_layer=0, lr_scale_change=1, lambda_tv=1, lambda_loss=1):
    # 生成画像のサイズ
    size = 512
    # style img array (preprocessed)
    style_img = image.load_img(style_path, target_size=(size, size))
    x_style = image.img_to_array(style_img)
    x_style = np.expand_dims(x_style, axis=0)
    x_style = preprocess_input(x_style)

    _, h_style, w_style, _ = tf.shape(x_style).numpy()
    # layerwise features of style image
    style_features = feature_model.predict(x_style)

    generated_image = np.random.rand(h_style*w_style*3) * 255
    generated_image = generated_image.reshape((1, h_style, w_style, 3))

    x_gen = preprocess_input(generated_image)
    x_gen = tf.Variable(x_gen, dtype=tf.float32)

    config = {
        'model': model,
        'image': x_gen,
        'style_features': style_features,
        'n_layer': n_layer,
        'lambda_tv': lambda_tv,
        'lambda_loss': lambda_loss
        }
    
    opt = tf.optimizers.Adam(learning_rate=10, beta_1=0.99, epsilon=1e-1)
    #opt = tf.optimizers.SGD(learning_rate=0.1)

    for i in range(n_iterations):
        with tf.GradientTape() as tape: 
            loss = compute_loss(**config)
        grads = tape.gradient(loss, config['image'])
         
        opt.apply_gradients([(grads, x_gen)])
        clipped = tf.clip_by_value(x_gen, -1, 1)
        x_gen.assign(clipped)
        
        if i%10==0:
            loss_value = loss.numpy()
            print(i, loss_value)
            plot_img = x_gen.numpy()
            plot_img = deprocess_img(plot_img)
            plt.imshow(plot_img)
            plt.show()
            # ここの実装適当。スクリプト化するときにきちんとする。
            imgs.append(plot_img)

    plot_img = x_gen.numpy()
    plot_img = deprocess_img(plot_img)

    return plot_img

# 3. `conv1_1`の特徴マップが運ぶスタイルの可視化

今回選んだVGG19の5つの層のうち、一番上流にある特徴マップのGram行列が担うneural styleを計算してみましょう。つまり、`n_layer=0`に対するneural styleを可視化します：

In [ ]:
neural_style = run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=0)

In [ ]:
plt.figure(dpi=200)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

Goghの《星月夜》の`conv1_1` neural styleは、この絵画の画風の高周波成分に対応していることが見て取れました。

# 4. `conv2_1`の特徴マップが運ぶスタイルの可視化

上流から数えて2番目にある特徴マップのGram行列が担うneural styleを計算してみましょう。つまり、`n_layer=1`に対するneural styleを可視化します：

In [ ]:
neural_style = run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=1, lambda_tv= 1, lambda_loss=1/1000)

In [ ]:
plt.figure(dpi=200)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

`conv2_1` neural styleはだいぶ大きな画風構造を捉えるようになってきて、画風の周波数は少し低周波側に寄ってきました。

# 5. `conv3_1`の特徴マップが運ぶスタイルの可視化

上流から数えて3番目にある特徴マップのGram行列が担うneural styleを計算してみましょう。つまり、`n_layer=2`に対するneural styleを可視化します：

In [ ]:
neural_style  =run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=2,lambda_tv= 1, lambda_loss=1/10000)

In [ ]:
plt.figure(dpi=200)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

`conv3_1` neural styleはタッチに近いような大きなスケールの画風に近いものになってきました。

# 6. `conv4_1`の特徴マップが運ぶスタイルの可視化

上流から数えて4番目にある特徴マップのGram行列が担うneural styleを計算してみましょう。つまり、`n_layer=3`に対するneural styleを可視化します：

In [ ]:
neural_style  =run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=3, lambda_tv= 2, lambda_loss=1/100000000)

In [ ]:
plt.figure(dpi=200)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

In [ ]:
neural_style = run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=3, lr_scale_change=2)

In [ ]:
plt.figure(dpi=150)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

`conv4_1` neural styleはタッチに近いような大きなスケールの画風に近いものになってきました。

# 7. `conv5_1`の特徴マップが運ぶスタイルの可視化

上流から数えて5番目にある特徴マップのGram行列が担うneural styleを計算してみましょう。つまり、`n_layer=4`に対するneural styleを可視化します：

In [ ]:
def compute_loss(model, image, style_features, n_layer, lambda_tv, lambda_loss):
    
    img_features = model(image)  
    img_style = img_features[n_layer]

    style_features = style_features[n_layer]
    gram_target = gram_matrix(style_features)

    style_loss =  get_style_loss(img_style, gram_target)
    tv_loss = total_variation_loss(image)
  
    return  (style_loss  + tv_loss * lambda_tv ) * lambda_loss

imgs = []

def run_neural_styler(style_path, model, n_iterations=1000, n_layer=0, lr_scale_change=1, lambda_tv=1, lambda_loss=1):
    # 生成画像のサイズ
    size = 256
    # style img array (preprocessed)
    style_img = image.load_img(style_path, target_size=(size, size))
    x_style = image.img_to_array(style_img)
    x_style = np.expand_dims(x_style, axis=0)
    x_style = preprocess_input(x_style)

    _, h_style, w_style, _ = tf.shape(x_style).numpy()
    # layerwise features of style image
    style_features = feature_model.predict(x_style)

    generated_image = np.random.rand(h_style*w_style*3) * 255
    generated_image = generated_image.reshape((1, h_style, w_style, 3))


    #generated_image = np.random.normal(1, 0.1, (1, h_style, w_style, 3)) * 255/2
    #generated_image = np.clip(generated_image, 0, 255)

    x_gen = preprocess_input(generated_image)
    x_gen = tf.Variable(x_gen, dtype=tf.float32)

    norm_means = np.array([103.939, 116.779, 123.68])
    min_vals = -norm_means
    max_vals = 255 - norm_means  

    config = {
        'model': model,
        'image': x_gen,
        'style_features': style_features,
        'n_layer': n_layer,
        'lambda_tv': lambda_tv,
        'lambda_loss': lambda_loss
        }
    
    #imgs = []
    opt = tf.optimizers.Adam(learning_rate=10, beta_1=0.99, epsilon=1e-1)
    #opt = tf.optimizers.SGD(learning_rate=0.1)


    for i in range(n_iterations):
        with tf.GradientTape() as tape: 
            loss = compute_loss(**config)
        grads = tape.gradient(loss, config['image'])
         
        opt.apply_gradients([(grads, x_gen)])
        clipped = tf.clip_by_value(x_gen, -1, 1)
        x_gen.assign(clipped)
        if i%10==0:
            loss_value = loss.numpy()
            print(i, loss_value)
            plot_img = x_gen.numpy()
            plot_img = deprocess_img(plot_img)
            plt.imshow(plot_img)
            plt.show()
            # ここの実装適当。スクリプト化するときにきちんとする。
            imgs.append(plot_img)

    plot_img = x_gen.numpy()
    plot_img = deprocess_img(plot_img)

    return plot_img

In [ ]:
neural_style = run_neural_styler('/content/drive/My Drive/style/Gogh.jpg', feature_model, n_iterations=1000, n_layer=4, lambda_tv=1, lambda_loss=1e-4)

In [ ]:
plt.figure(dpi=200)
plt.imshow(imgs[-1])
plt.axis('off')
plt.show()

`conv5_1` neural styleはタッチに近いような大きなスケールの画風に近いものになってきました。

## このコードで学んだDLコーディング関係こと

- <font color=darkorange>TensorFlow2</font>において自動微分を計算するための勾配テープの使い方（tf.GradientTape API）。

- その勾配で直接的に勾配降下をさせる方法（`optimizer.apply_apply_gradients([(grads, variables)])`）

詳しくは

https://amazon.co.jp/dp/4873119286/

の12.3.8と12.3.9。

# 問題
スタイル画像の縦横比を正方形に調整する必要はありません。[Gatys et al., Image Style Transfer Using Convolutional Neural Networks]のFigure.1の図の結果のように、星月夜画像の縦横比を維持するように実装を修正して見ましょう。

# 問題
[Gatys et al., Image Style Transfer Using Convolutional Neural Networks]Figure.1に合わせて、VGG19の‘conv1 2’, ‘conv2 2’, ‘conv3 2’, ‘conv4 2’, ‘conv5 2’でやって見ましょう。

# 問題
学習率スケジューリングを実装することで、この最適化計算がもう少し改善できるでしょうか？

# 問題
全く同じものをPytorchで実装して見ましょう。